In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from keras.initializers import glorot_uniform

# What is ResNet?
ResNet, short for Residual Networks is a classic neural network used as a backbone for many computer vision tasks. This model was the winner of ImageNet challenge in 2015. The fundamental breakthrough with ResNet was it allowed us to train extremely deep neural networks with 150+layers successfully.

# Residual Blocks

x is input, y=F(x) is output,

The layers in a traditional network are learning the true output H(x).

y = x + F(x)  =>  F(x) = y - x   the layers in a residual network are learning the residual F(x).

![](./Sources/FX1.png)

**Identity Block** is used when there is no change in input and output dimensions. **Convolutional block** is almost same as identity block but there is a convolutional layer in short-cut path to just change the dimension such that the dimension of input and output matches.

![](./Sources/both.png)

## Identity Block

In [2]:

def identity_block(X, f, filters, stage, block):
    """ 
    Arguments:
    X -- input of shape (m, height, width, channel)
    f -- shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base =  'id_S'+ str(stage) +'_B'+ block+'_L' 
    bn_name_base = 'bn_S' + str(stage) +'_B'+ block+'_L'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Saving the input value.we need this later to add to the output. 
    X_shortcut = X
    
    # First layer
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '1')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '1')(X)
    X = Activation('relu')(X)

    
    # Second layer
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2')(X)
    X = Activation('relu')(X)

    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '3')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '3')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)   
    
    return X

## Convolutional Block

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """ 
    Arguments:
    X -- input of shape (m, height, width, channel)
    f -- shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base =  'id_S'+ str(stage) +'_B'+ block+'_L' 
    bn_name_base = 'bn_S' + str(stage) +'_B'+ block+'_L'
    
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    # First layer 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '1')(X) # 1,1 is filter size
    X = BatchNormalization(axis = 3, name = bn_name_base + '1')(X)  # normalization on channels
    X = Activation('relu')(X)

      
    # Second layer  
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2')(X)
    X = Activation('relu')(X)


    # Third layer
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '3')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '3')(X)


    # Skip connection 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1_Skip_connection')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1_Skip_connection')(X_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
       
    return X

# Creating ResNet 50

![](./Sources/ResNet50.png)

In [4]:

def ResNet50(input_shape=(64, 64, 3), classes=3):
    
    """ 
    ResNet50 includes 4 main stages, each having a number of blocks, 
    the initial block of each stage is a convolutional_block to modify the input size
    """

    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input) #3,3 padding

    # Stage 0
    X = Conv2D(64, (7, 7), strides=(2, 2), name='S0')(X) 
    X = BatchNormalization(axis=3, name='bn_S0')(X) 
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X) 

    # Stage 1
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=1, block='1', s=1)   
    X = identity_block(X, 3, [64, 64, 256], stage=1, block='2') 
    X = identity_block(X, 3, [64, 64, 256], stage=1, block='3')


    # Stage 2
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 2, block='1', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=2, block='2')
    X = identity_block(X, 3, [128, 128, 512], stage=2, block='3')
    X = identity_block(X, 3, [128, 128, 512], stage=2, block='4')

    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 3, block='1', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=3, block='2')
    X = identity_block(X, 3, [256, 256, 1024], stage=3, block='3')
    X = identity_block(X, 3, [256, 256, 1024], stage=3, block='4')
    X = identity_block(X, 3, [256, 256, 1024], stage=3, block='5')
    X = identity_block(X, 3, [256, 256, 1024], stage=3, block='6')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 4, block='1', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='2')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='3')

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (64, 64, 3), classes = 3)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
S0 (Conv2D)                     (None, 32, 32, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_S0 (BatchNormalization)      (None, 32, 32, 64)   256         S0[0][0]                         
___________________________________________________________________________________________